# Object Oriented Programming (Classes)

Object Oriented Programming is about how we *organize* our ideas in code.

Programs are made up of two fundamental, conceptual components:
    
  - Data
  - Algorithms to manipulate the data

So to have an expressive and useful programming language, we need ways to both

  - Create new types of data.
  - Create re-usable algorithms to manipulate that data.

Sometimes the algorithms we need to manipulate data are tied closely to the data itself, and in this case we would like to

  - Associate algorithms with specific data structures

### Basic Example: Lists

Python `list`s and `dictionary`s are a very useful type of data structure, and they have lot's of associated algorithms 

In [ ]:
PI = 3.14159

In [7]:
lst = [1, 2, 2, 3, 4, 4, 4]

# Associated algorithm: count
print(lst.count(2))
print(lst.count(3))

print(lst)

2
1
[1, 2, 2, 3, 4, 4, 4]


The `count` function is assoicated with the `list` data type.

Functions that are associated to a specific data type in this way are called *methods*.  So we would say

> `count` is a method of the data type `list`

Methods are (generally) called using the `.` notataion: `data_element.method(additional_arguments)`.

Some methods actually *change* the data they operate on:

In [8]:
print(lst)

lst.append(5)

print(lst)

[1, 2, 2, 3, 4, 4, 4]
[1, 2, 2, 3, 4, 4, 4, 5]


Methods which do **not** change the underlying data (`list.count`) are called **pure methods**, methods that *do* change the underlying data (`list.append`) are called **impure methods**.

Some things that do not look like methods actually are, indexing for example:

In [9]:
print(lst[2])

print(lst.__getitem__(2))

2
2


The `__getitem__` is called a **magic method**.  There are spelled with two underscores and can be called with special syntax.

In [10]:
# lst[2] = 100
lst.__setitem__(2, 100)

# lst[2]
print(lst.__getitem__(2))

# len(lst)
print(lst.__len__())

# lst[1:5]
print lst.__getslice__(1, 5)

100
8
[2, 100, 3, 4]


### More Advanced Example: Default Dict

The python standard library has many examples of additional data types.  We will be re-implementing two of the more useful ones, `defaultdict` and `OrderedDict`.

In [2]:
from collections import defaultdict, OrderedDict

`defautdict` is a simple but effective alternative to a dictionary.

Recall that with a normal dictionary, attempting to lookup a key that does not exist is an error.

In [18]:
D = {'a': 1, 'b': 2}

D['c']

KeyError: 'c'

A `defaultdict` allows you to specify a default value to return when a non-existant key lookup is attempted.

In [26]:
D = defaultdict(lambda: 0, {'a': 1, 'b': 2})

In [27]:
D['a']

1

In [28]:
D['c']

0

In [29]:
print(D)

defaultdict(<function <lambda> at 0x104773050>, {'a': 1, 'c': 0, 'b': 2})


It's a bit weird to have to pass in a function that returns the defualt value instead of the default value itself, but this is needed to avoid weird problems arising from mutable objects like lists.

In summary, this works as intended:

In [3]:
D = defaultdict(list, {})

print(D['a'])

D['a'].append(1) #appends to a list that wasn't there before
D['a'].append(2)
D['b'].append(1)

print(D)

[]
defaultdict(<type 'list'>, {'a': [1, 2], 'b': [1]})


### Making Your Own Default Dict

Let's make our own default dictionaries.

There are two concepts we need

  - A `class` is a template for a new data type.  It contains inforamtion on what data is needed to constrict the data type, how to store the data internally, and what algorithms can be applied to the data type.
  - An instance of a class is a concrete object of the new data type.
  
A class is a recipe for constructing instances of that class.

#### Example of a Class: defaultdict

`defaultdict` is a class

In [33]:
from inspect import isclass
isclass(defaultdict)

True

Using the class `defaultdict` as a function creates an instance of that class.

In [34]:
D = defaultdict(lambda: 0, {'a': 1, 'b': 2})
isinstance(D, defaultdict)

True

We usually abbreviate the phrase

> `D` is an instance of class `defaultdict`.

as

> `D` is a `defaultdict`.

#### How to Store Data in a Class

The basics of creating a custom class in python is very easy

In [39]:
class MyClass(object):
    pass

In [37]:
my_instance = MyClass()

In [38]:
isinstance(my_instance, MyClass)

True

This is a pretty dumb class as it stands, it cant really *do* anything.  To get something useful we have to add data and behaviour to out class.

The first step is to determine what data we need to store.  In this case it's pretty easy, we need

  - The underlying dictionary that we are going to attempt lokups into.
  - The default action to take when a lookup fails.

Let's mimic the way python's built in default dict works.  We need to add some functionality to **supply and then store** both of these data elements when we create an instance of the class.  This is done using a special *method*, `__init__`.

In [4]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary

There's a lot of new concepts in this code, but let's first see how it works.

In [5]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})
print(MD.default)
print(MD.default())
print(MD.dictionary)

<function <lambda> at 0x10ec6db90>
0
{'a': 1, 'b': 2}


In [6]:
MD.default(0)

TypeError: <lambda>() takes no arguments (1 given)

In [7]:
MD.dictionary

{'a': 1, 'b': 2}

When we use a class, it is to create *instances of that class*, which we then work with.  We very rarely work with the class directly, and we will often be working with more than one instance of a single class.

In [50]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})
MD2 = MyDefaultDict(lambda: 1, {'a': 2, 'b': 3, 'c': 5})

print(MD.dictionary)
print(MD2.dictionary)

{'a': 1, 'b': 2}
{'a': 2, 'c': 5, 'b': 3}


Inside of the code defining a class, `self` represents the instance of the class we are manipulating.

A statement like

```
self.default = default
```

creates what is known as an *instance varaible* or *instance data*.  In this specific line, we attach the `default` function to the current instance of the class.

In this way, once created, each instance of `MyDefaultDict` stores both `default` and `dictionary` data.

#### Addding Methods to Manipulate Data in a Class

Let's implement `__getitem__` and `__setitem__`, which will allow us to index into instances of our class like this

```
MD['a']
# Means the same thing as MD.__getitem__('a')

MD['c'] = 3
# Means the same thing as MD.__setitem__('c', 3)
```

As a first attempt, let's ignore our goal of adding default behaviour, we can add that later on down the line.

In [70]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary
        
    def __getitem__(self, key):
        return self.dictionary[key]
    
    def __setitem__(self, key, value):
        self.dictionary[key] = value

Let's test it out.

In [75]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})

print(MD['a'])
print(MD['b'])

MD['c'] = 3

print(MD.dictionary)

1
2
{'a': 1, 'c': 3, 'b': 2}


In [63]:
type(MD.dictionary)

dict

#### Adding the Special Default Behaviour

In [77]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary
        
    def __getitem__(self, key):
        if key in self.dictionary:
            return self.dictionary[key]
        else:
            self.dictionary[key] = self.default()
            return self.dictionary[key]
    
    def __setitem__(self, key, value):
        self.dictionary[key] = value

In [78]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})

print(MD['a'])
print(MD['b'])
print(MD['c'])
print(MD.dictionary)

1
2
0
{'a': 1, 'c': 0, 'b': 2}


#### Adding Other Dict-y Things

A few things that should work for dictionaries still don't work for our new datatype

In [79]:
len(MD)

TypeError: object of type 'MyDefaultDict' has no len()

Additionally, code like

```
'c' in MD
```

and

```
for key in MD:
    print key, MD[key]
```

will cause an infinite loop, due to a design error in python itself.

Let's fix that with more magic methods.

In [1]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary
        
    def __getitem__(self, key):
        if key in self.dictionary:
            return self.dictionary[key]
        else:
            return self.default()
    
    def __setitem__(self, key, value):
        self.dictionary[key] = value
        
    def __len__(self):
        return len(self.dictionary)
        
    def __contains__(self, key):
        return key in self.dictionary
    
    def __iter__(self):
        for key in self.dictionary:
            yield key

In [2]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})

In [3]:
print('a' in MD)

True


In [4]:
for key in MD:
    print key, MD[key]

a 1
b 2


#### Non-Magic Methods

It's worth mentioning that not all methods are magic.  Here is a class that represents a simple quadratic polynomial, and has an `evaluate` method, which plugs a number into the polynomial.

In [45]:
class QuadraticPolynomial(object):
    """A class representing a polynomial like:
    
        a_0 + a_1 x + a_2 x^2
    """
    
    def __init__(self, a0, a1, a2):
        self.coefficients = (a0, a1, a2)
        
    def evaluate(self, x):
        a0, a1, a2 = self.coefficients
        return a2 * x * x + a1 * x + a0

#### Excercizes

1. Use the `__add__` magic method to allow something like `QuadraticPolynomial(1, 1, 1) + QuadraticPolynomial(1, 0, 1)`.  The new method should *return* another `QuadraticPolynomial`.

2. Write a class `LinearPolynomial`.  Add a method `differentiate` to `QuadraticPolynomial` that returns a `LinearPolynomial`.

3. Suppose we create a new `QuadraticPolynomial` like `QuadraticPolynomial(1, 1, 0)`.  Is this really a `QuadraticPolynomial`?  What should it be?  How can you resolve this weird inconsistency in data types?

In [46]:
class QuadraticPolynomial(object):
    """A class representing a polynomial like:
    
        a_0 + a_1 x + a_2 x^2
    """
    
    def __init__(self, a0, a1, a2):
        self.coefficients = (a0, a1, a2)
        
    def evaluate(self, x):
        a0, a1, a2 = self.coefficients
        return a2 * x * x + a1 * x + a0
    
    def __repr__(self):
        return "QuadraticPolynomial({}, {}, {})".format(
            self.coefficients[0],
            self.coefficients[1],
            self.coefficients[2])
    
    def __str__(self):
        return "{}x^2 + {}x + {}".format(
            self.coefficients[2],
            self.coefficients[1],
            self.coefficients[0])
    
    def _apply_operation_to_coefficients(self, other, operation):
        a0 = operation(self.coefficients[0], other.coefficients[0])
        a1 = operation(self.coefficients[1], other.coefficients[1])
        a2 = operation(self.coefficients[2], other.coefficients[2])
        return QuadraticPolynomial(a0, a1, a2)
    
    def __add__(self, other):
        return self._apply_operation_to_coefficients(other, lambda x, y: x + y)
    
    def __sub__(self, other):
        return self._apply_operation_to_coefficients(other, lambda x, y: x - y)
    
    def differentiate(self):
        a0, a1, a2 = self.coefficients
        return LinearPolynomial(a1, 2 * a2)

In [47]:
class LinearPolynomial(object):
    
    def __init__(self, a0, a1):
        self.coefficients = (a0, a1)
    
    def __str__(self):
        return "{}x + {}".format(self.coefficients[1], self.coefficients[0])

In [48]:
def polynomial_factory(coefficients):
    if coefficients[2] == 0:
        return LinearPolynomial(coefficients[0],
                                coefficients[1])
    else:
        return QuadraticPolynomial(coefficients[0],
                                   coefficients[1],
                                   coefficients[2])